# 📊 Introduction to Binomial Trees for Option Pricing

## Overview
In this notebook, we explore the use of **binomial trees** for pricing options, based on the principles outlined in Chapter 11 of John Hull's *Options, Futures, and Other Derivatives*. The binomial tree method is a flexible and powerful tool for pricing European and American options, and it allows for adjustments in various parameters, including volatility, time to expiration, and the nature of the underlying asset.

## 🌳 What is a Binomial Tree?
A binomial tree is a discrete-time model for the evolution of the price of an underlying asset over time. It provides a step-by-step method to model the potential future movements of the asset price, where at each step, the price can move **up** or **down** by certain factors.

### Key Features:
- **Time-Stepping**: The life of the option is divided into equal time intervals, and at each interval, the underlying asset can move up or down.
- **Risk-Neutral Valuation**: The model assumes a **risk-neutral world**, where expected returns are the same as the risk-free rate, simplifying the valuation of derivatives.
- **Flexibility**: Binomial trees are highly flexible and can handle different payoff structures and American options, which can be exercised at any point before expiration.

## ⚙️ Key Parameters
In a binomial model, the following parameters are essential for constructing the tree:
- **u**: The upward movement factor.
- **d**: The downward movement factor.
- **p**: The probability of an upward movement, based on risk-neutral probabilities.
- **r**: The risk-free interest rate.
- **Δt**: The time step, which is the total time to maturity divided by the number of steps in the tree.

The **upward factor** ($u$) and **downward factor** ($d$) are given by:
$$
u = e^{\sigma \sqrt{\Delta t}}
$$
$$
d = e^{-\sigma \sqrt{\Delta t}}
$$
Where:
- $\sigma$ is the volatility of the underlying asset.
- $\Delta t$ is the time interval between each step.

The **risk-neutral probability** ($p$) is calculated as:
$$
p = \frac{e^{r \Delta t} - d}{u - d}
$$

## 🧮 Option Valuation Using Binomial Trees
To value an option using a binomial tree:
1. **Construct the tree** of possible asset prices at each time step.
2. **Calculate the option payoff** at each final node (at maturity).
3. **Work backwards** through the tree, discounting the option value at each node using the risk-neutral probabilities until the current time is reached.

For an **American option**, early exercise is evaluated at each node, comparing the value of exercising versus holding the option.

## 🚀 Advantages of the Binomial Model
- Can be used for both **European** and **American options**.
- Handles situations where the option can be exercised before expiration.
- Provides flexibility in modeling varying dividends and volatility.

In [2]:
import numpy as np

In [3]:
# Example 
"""
Pricing an amercian put.
"""
spot = 50
strike = 52
rf = 0.05
vol = 0.3
dt = 1 # Option maturity = 2y and we want to compute a tree with two branch

# First we estimate p
u = np.exp(vol * np.sqrt(dt))
d = np.exp(-vol * np.sqrt(dt))

p = (np.exp(rf * dt) - d) / (u - d)
print(f'Parameter p : {p}')

first_branch = [spot * u, spot * d]
second_branch = [first_branch[0] * u, first_branch[0] * d, first_branch[1] * u, first_branch[1] * d]
print(f'First branch : {first_branch}')
print(f'Second branch : {second_branch}')

put_value_b2 = [max(strike - second_branch[0], 0), max(strike - second_branch[1], 0), max(strike - second_branch[2], 0), max(strike - second_branch[3], 0)]
put_value_b1 = [p * put_value_b2[0] + (1 - p) * put_value_b2[1], p * put_value_b2[2] + (1 - p) * put_value_b2[3]]
put_value_b0 = p * put_value_b1[0] + (1 - p) * put_value_b1[1]

print(f'The put value at t0 is : {put_value_b0}')

Parameter p : 0.5097408651817704
First branch : [67.49294037880016, 37.040911034085894]
Second branch : [91.10594001952546, 50.0, 50.00000000000001, 27.440581804701324]
The put value at t0 is : 6.9025753364216245


***Problems & Exercises***

In [11]:
# 13.11
"""
Pricing an European call.
"""
spot = 100
u_d = 0.1
rf = 0.08
periods = 2 
maturity = 1
K = 100

# We'll make a function so we can use it for different exercises
def binomial_tree(option_type: str, spot: float, strike: float, risk_free: float, maturity: int, number_of_branch: int, sigma: float=None, up: float=None, down: float=None):
    # First we prevent any wrong input
    if ((not u_d) and (not sigma)) or (u_d and sigma):
        print('You should either put u_d or sigma')
        return
    
    # Then, we compute the parameters
    dt = maturity / number_of_branch
    
    if sigma:
        u = np.exp(sigma * np.sqrt(dt))
        d = np.exp(-sigma * np.sqrt(dt))
    else:
        u = 1 + up
        d = 1 - down
    
    p = (np.exp(risk_free * dt) - d) / (u - d)
    
    # We model the tree as a matrix (the tree grows in the upper triangle)
    tree = np.zeros((number_of_branch + 1, number_of_branch + 1))
    tree[0, 0] = spot
    for i in range(1, number_of_branch + 1):
        tree[i, 0] = tree[i - 1, 0] * u 
        for j in range(1, i + 1):
            tree[i, j] = tree[i - 1, j - 1] * d
    
    # Calculate the option values at maturity
    option_values = np.zeros((number_of_branch + 1))
    for j in range(number_of_branch + 1):
        if option_type.lower() == 'call':
            option_values[j] = max(0, tree[number_of_branch, j] - strike)
        elif option_type.lower() == 'put':
            option_values[j] = max(0, strike - tree[number_of_branch, j])
    
    # Backward induction to get the option price at time 0
    for i in range(number_of_branch - 1, -1, -1):
        for j in range(i + 1):
            option_values[j] = (p * option_values[j] + (1 - p) * option_values[j + 1]) * np.exp(-risk_free * dt)
    
    return option_values[0]
    
# Now we can price our european call
call_price = binomial_tree(option_type='call', spot=spot, strike=K, risk_free=rf, maturity=maturity, number_of_branch=periods, up=u_d, down=u_d)
print(f'the european call value is : {call_price}')

the european call value is : 9.609206301971435


In [12]:
# 13.12 
"""
Same exercise but with a put. 
"""
put_price = binomial_tree(option_type='put', spot=spot, strike=K, risk_free=rf, maturity=maturity, number_of_branch=periods, up=u_d, down=u_d)
print(f'the european call value is : {put_price}')

# Check for call-put parity
if put_price + spot == call_price + K * np.exp(-rf * maturity):
    print('Call-Put Parity is respected')

the european call value is : 1.9208409406350109
Call-Put Parity is respected


In [16]:
# 13.17 
"""
European call pricing.
"""
spot = 50
u = 0.06
d = 0.05
rf = 0.05
periods = 2
maturity = 0.5
K = 51

call_price = binomial_tree(option_type='call', spot=spot, strike=K, risk_free=rf, maturity=maturity, number_of_branch=periods, up=u, down=d)
print(f'the european call value is : {call_price}')

the european call value is : 1.6350711385184145


In [19]:
# 13.18 
"""
Just pricing the put.
"""
K = 51
put_price = binomial_tree(option_type='put', spot=spot, strike=K, risk_free=rf, maturity=maturity, number_of_branch=periods, up=u, down=d)
print(f'the european call value is : {put_price}')

the european call value is : 1.37587665196338


***Since we have a created a function, the other exercises are not really interesting***